In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline
import seaborn as sns
plt.rc('font',family='Malgun Gothic')
plt.rc('axes',unicode_minus=False)
import sys, warnings
warnings.filterwarnings("ignore")
import re
import os
import datetime
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, KFold, cross_val_predict, RandomizedSearchCV, GridSearchCV, cross_val_score
from xgboost import XGBClassifier, plot_importance
from lightgbm import LGBMClassifier # 한글 지원 X
from vecstack import stacking, StackingTransformer
from sklearn.metrics import accuracy_score, roc_auc_score
from mlxtend.plotting import plot_decision_regions
from IPython.display import display
import itertools
pd.set_option('display.max_columns', 1200)
np.set_printoptions(threshold = sys.maxsize)
from scipy.stats.mstats import gmean
from scipy.stats import randint as sp_randint
from scipy.stats import uniform
import random
from sklearn.dummy import DummyClassifier
from sklearn import metrics
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA

In [2]:
tr = pd.read_csv('X_train.csv',encoding='cp949')

In [3]:
tr

,cust_id,tran_date,store_nm,goods_id,gds_grp_nm,gds_grp_mclas_nm,amount
0,0,2007-01-19 00:00:00,강남점,127105,기초 화장품,화장품,850000
1,0,2007-03-30 00:00:00,강남점,342220,니 트,시티웨어,480000
2,0,2007-03-30 00:00:00,강남점,127105,기초 화장품,화장품,3000000
3,0,2007-03-30 00:00:00,강남점,342205,니 트,시티웨어,840000
4,0,2007-03-30 00:00:00,강남점,342220,상품군미지정,기타,20000
...,...,...,...,...,...,...,...
231999,3499,2007-12-17 00:00:00,본 점,127129,상품군미지정,기타,-135000
232000,3499,2007-12-23 00:00:00,노원점,285136,시티웨어,시티웨어,6380000
232001,3499,2007-12-23 00:00:00,노원점,39107,야채,농산물,40800
232002,3499,2007-12-27 00:00:00,본 점,740120,어덜트,명품,4880000


In [4]:
y = pd.read_csv('y_train.csv',encoding='cp949')

In [6]:
tr = tr.merge(y)

In [17]:
a = tr.groupby('gender')['store_nm'].agg('unique').reset_index()

In [14]:
tr.store_nm.unique()

array(['강남점', '잠실점', '관악점', '광주점', '본  점', '일산점', '분당점', '창원점', '부산본점',
       '대전점', '미아점', '영등포점', '노원점', '동래점', '청량리점', '센텀시티점', '안양점', '울산점',
       '부평점', '포항점', '인천점', '대구점', '전주점', '상인점'], dtype=object)

In [21]:
for i in a['store_nm'][0]:
    if i in tr.store_nm.unique():
        print(i)

강남점
잠실점
본  점
일산점
분당점
창원점
관악점
부산본점
미아점
영등포점
노원점
동래점
광주점
센텀시티점
대전점
청량리점
부평점
안양점
포항점
인천점
울산점
대구점
전주점
상인점


In [22]:
data = pd.read_csv('features0521.csv',encoding='cp949')

In [27]:
data

,cust_id,총구매액,구매건수,평균구매액,최대구매액,구매상품종류1,구매상품종류2,구매상품종류3,내점일수,구매주기,주말방문비율,봄-구매비율,여름-구매비율,가을-구매비율,겨울-구매비율,환불금액,환불건수,내점당구매액,내점당구매건수,최근3개월_구매금액,최근3개월_구매건수,최근6개월_구매금액,최근6개월_구매건수,최근12개월_구매금액,최근12개월_구매건수,구매추세,가격선호도,시즌선호도,휴면(이탈)여부,기타_구매금액,기타_구매건수,기타_구매여부,농산물_구매금액,농산물_구매건수,농산물_구매여부,가공식품_구매금액,가공식품_구매건수,가공식품_구매여부,축산가공_구매금액,축산가공_구매건수,축산가공_구매여부,수산품_구매금액,수산품_구매건수,수산품_구매여부,화장품_구매금액,화장품_구매건수,화장품_구매여부,디자이너_구매금액,디자이너_구매건수,디자이너_구매여부,시티웨어_구매금액,시티웨어_구매건수,시티웨어_구매여부,일용잡화_구매금액,일용잡화_구매건수,일용잡화_구매여부,육류_구매금액,육류_구매건수,육류_구매여부,주구매상품,주구매지점,상품별구매순서,여부3,구매건수_많은달,구매건수_많은계절,월말_평균구매금액,월중반_평균구매금액,월초_평균구매금액,월말_구매건수,월중반_구매건수,월초_구매건수,금_평균구매금액,목_평균구매금액,수_평균구매금액,월_평균구매금액,일_평균구매금액,토_평균구매금액,화_평균구매금액,금_구매건수,목_구매건수,수_구매건수,월_구매건수,일_구매건수,토_구매건수,화_구매건수,다른지점구매비율,상품군미지정_구매금액,상품군미지정_구매건수,상품군미지정_구매여부,농산가공_구매금액,농산가공_구매건수,농산가공_구매여부,유제품_구매금액,유제품_구매건수,유제품_구매여부,일반가공식품_구매금액,일반가공식품_구매건수,일반가공식품_구매여부,기초 화장품_구매금액,기초 화장품_구매건수,기초 화장품_구매여부,청과_구매금액,청과_구매건수,청과_구매여부,디자이너부틱_구매금액,디자이너부틱_구매건수,디자이너부틱_구매여부,야채_구매금액,야채_구매건수,야채_구매여부,수입식품_구매금액,수입식품_구매건수,수입식품_구매여부,유기농야채_구매금액,유기농야채_구매건수,유기농야채_구매여부,소분류주구매상품
0,0,68282840,74,922741,11264000,35,28,17,19,17,0.527027,0.094595,0.391892,0.378378,0.135135,6860000.0,3.0,3.593834e+06,3.894737,29084140.0,27.0,42601240.0,61.0,68282840,74,0.098039,3,1,0,507200.0,13.0,1.0,74360.0,3.0,1.0,35520.0,1.0,1.0,119000.0,2.0,1.0,866000.0,4.0,1.0,22450000.0,7.0,1.0,11264000.0,4.0,1.0,9806000.0,7.0,1.0,0.0,0.0,0.0,567860.0,10.0,1.0,기타,강남점,화장품,0,10,summer,1.032426e+06,6.464486e+05,1.113962e+06,13,28,33,1.890756e+06,610200.0,-3.647033e+05,412440.0,2.338358e+06,6.281607e+05,2.243000e+06,9,6,12,6,12,27,2,0.040541,507200.0,13.0,1.0,0.0,0.0,0.0,119000.0,2.0,1.0,35520.0,1.0,1.0,22450000.0,7.0,1.0,74360.0,3.0,1.0,11264000.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,상품군미지정
1,1,2136000,3,712000,2136000,2,2,2,2,1,0.000000,0.000000,0.000000,1.000000,0.000000,300000.0,1.0,1.068000e+06,1.500000,2136000.0,3.0,2136000.0,3.0,2136000,3,0.250000,2,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,스포츠,잠실점,스포츠,1,10,fall,7.120000e+05,0.000000e+00,0.000000e+00,3,0,0,0.000000e+00,918000.0,0.000000e+00,300000.0,0.000000e+00,0.000000e+00,0.000000e+00,0,2,0,1,0,0,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,아웃도아
2,2,3197000,4,799250,1639000,1,2,2,2,1,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,0.0,1.598500e+06,2.000000,0.0,0.0,0.0,0.0,3197000,4,0.250000,1,3,1,139000.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,기타,관악점,남성 캐주얼,0,1,winter,0.000000e+00,0.000000e+00,7.992500e+05,0,0,4,0.000000e+00,0.0,7.417500e+05,856750.0,0.000000e+00,0.000000e+00,0.000000e+00,0,0,2,2,0,0,0,0.000000,139000.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,시티캐주얼
3,3,16077620,44,365400,4935000,24,20,13,18,16,0.318182,0.590909,0.159091,0.113636,0.136364,0.0,0.0,8.932011e+05,2.444444,549620.0,4.0,1284060.0,11.0,16077620,44,0.125000,2,0,0,396630.0,14.0,1.0,1599380.0,6.0,1.0,1346550.0,6.0,1.0,45000.0,1.0,1.0,400560.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,385900.0,2.0,1.0,237000.0,2.0,1.0,기타,광주점,캐주얼,0,3,spring,3.839627e+05,2.796327e+05,4.023456e+05,15,11,18,5.003333e+05,1081024.0,1.915167e+05,50308.0,1.053580e+06,6.200000e+04,1.768145e+05,3,5,6,5,5,9,11,0.000000,396630.0,14.0,1.0,120000.0,1.0,1.0,45000.0,1.0,1.0,1326550.0,5.0,1.0,0.0,0.0,0.0,46380.0,2.0,1.0,0.0,0.0,0.0,3000.0,1.0,1.0,187000.0,2.0,1.0,0.0,0.0,0.0,상품군미지정
4,4,29050000,3,9683333,24000000,2,3,3,2,85,0.000000,0.333333,0.000000,0.666667,0.000000,0.0,0.0,1.452500e+07,1.500000,27000000.0,2.0,27000000.0,2.0,29050000,3,0.500000,1,2,1,3000000.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2050000.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,화장품,본 점,화장품,1,11,fall,2.050000e+06,1.350000e+07,0.000000e+00,1,2,0,0.000000e+00,0.0,1.350000e+07,2050000.0,0.000

In [34]:
train = data[:3500].merge(y)

In [35]:
gender_mean = train.groupby('주구매상품')['gender'].mean()
data['주구매상품'] = data['주구매상품'].map(gender_mean)

In [36]:
gender_mean = train.groupby('주구매지점')['gender'].mean()
data['주구매지점'] = data['주구매지점'].map(gender_mean)

In [37]:
gender_mean = train.groupby('상품별구매순서')['gender'].mean()
data['상품별구매순서'] = data['상품별구매순서'].map(gender_mean)

In [38]:
gender_mean = train.groupby('구매건수_많은계절')['gender'].mean()
data['구매건수_많은계절'] = data['구매건수_많은계절'].map(gender_mean)

In [39]:
gender_mean = train.groupby('구매건수_많은달')['gender'].mean()
data['구매건수_많은달'] = data['구매건수_많은달'].map(gender_mean)

In [40]:
gender_mean = train.groupby('소분류주구매상품')['gender'].mean()
data['소분류주구매상품'] = data['소분류주구매상품'].map(gender_mean)

In [41]:
# 평균구매액 피처 제거
data = data.drop(columns=['평균구매액'])
# 구매주기, 주말방문비율, 계절별 구매비율 피처 제거
data = data.drop(columns=['구매주기','주말방문비율','봄-구매비율','여름-구매비율','가을-구매비율','겨울-구매비율'])
# 내점당구매액, 내점당구매건수 피처 제거
data = data.drop(columns=['내점당구매액','내점당구매건수'])
# 가격선호도, 시즌선호도 피처 제거
data = data.drop(columns=['가격선호도','시즌선호도'])
# 기타제품의 구매여부 피처 제거
data = data.drop(columns=['기타_구매여부'])
# 일자별 평균구매금액에서의 피처 제거
data = data.drop(columns=['월중반_평균구매금액','월초_평균구매금액'])
# 요일별 평균구매금액에서의 피처 제거
data = data.drop(columns=['월_평균구매금액','일_평균구매금액','토_평균구매금액'])

In [49]:
X = data[:3500]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y['gender'],test_size=0.3)

In [50]:
from xgboost import XGBClassifier

In [52]:
xgb = XGBClassifier()
xgb.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)